In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
df_districts = pd.read_csv('gb-districts.csv')
df_districts.head()

,Unnamed: 0,postal_code,postal_code_trimmed,state,postal_code_prefix,uk_region,postal_code_district
0,0,PH44,PH,Scotland,PH,Scotland,Perth
1,1,LE13,LE,England,LE,East Midlands,Leicester
2,2,LE14,LE,England,LE,East Midlands,Leicester
3,3,WR11,WR,England,WR,West Midlands,Worcester
4,4,BA8,BA,England,BA,South West,Bath


In [3]:
df_tax = pd.read_csv('../gb-tax-band-data.csv', encoding='latin-1')
df_tax.head()

,Geography1,Area Name,Band,Bungalow,Flat,Terraced,Semi-Detached,Detached
0,ENGWAL,ENGLAND AND WALES,All,"2,436,910","5,564,180","6,757,790","6,051,520","3,939,450"
1,ENGWAL,ENGLAND AND WALES,A,"342,430","2,193,790","2,288,430","989,130","17,090"
2,ENGWAL,ENGLAND AND WALES,B,"382,880","1,300,590","1,687,660","1,481,710","61,600"
3,ENGWAL,ENGLAND AND WALES,C,"647,190","1,123,910","1,415,450","1,831,410","420,870"
4,ENGWAL,ENGLAND AND WALES,D,"546,710","531,840","843,770","1,012,600","914,910"


In [4]:
df_districts['uk_region'].unique()

array(['Scotland', 'East Midlands', 'West Midlands', 'South West',
       'East of England', 'South East', 'North West', 'Wales',
       'Greater London', 'East England', 'North East', 'Northern Ireland'],
      dtype=object)

In [5]:
df_districts.loc[df_districts['postal_code_prefix'].isin(['DN','HD','HG','HU','HX','LS','S','WF','YO']), 'uk_region'] = 'Yorkshire and the Humber'

In [6]:
df_districts.drop(columns=['Unnamed: 0'], inplace=True)

In [7]:
df_districts

,postal_code,postal_code_trimmed,state,postal_code_prefix,uk_region,postal_code_district
0,PH44,PH,Scotland,PH,Scotland,Perth
1,LE13,LE,England,LE,East Midlands,Leicester
2,LE14,LE,England,LE,East Midlands,Leicester
3,WR11,WR,England,WR,West Midlands,Worcester
4,BA8,BA,England,BA,South West,Bath
...,...,...,...,...,...,...
43528,BA12,BA,England,BA,South West,Bath
43529,BA9,BA,England,BA,South West,Bath
43530,TR4,TR,England,TR,South West,Truro
43531,TR26,TR,England,TR,South West,Truro


In [8]:
df_districts['uk_region'].unique()

array(['Scotland', 'East Midlands', 'West Midlands', 'South West',
       'East of England', 'South East', 'North West', 'Wales',
       'Greater London', 'East England', 'Yorkshire and the Humber',
       'North East', 'Northern Ireland'], dtype=object)

In [9]:
filter = df_tax['Geography1'] == 'REGL'

In [10]:
df_tax[filter]['Area Name'].unique()

array(['NORTH EAST', 'NORTH WEST', 'YORKSHIRE AND THE HUMBER',
       'EAST MIDLANDS', 'WEST MIDLANDS', 'EAST', 'LONDON', 'SOUTH EAST',
       'SOUTH WEST'], dtype=object)

In [11]:
df_tax = df_tax[filter]

In [12]:
df_tax = df_tax.replace({'Area Name':{'NORTH EAST':'North East', 'NORTH WEST':'North West', 'YORKSHIRE AND THE HUMBER':'Yorkshire and the Humber', 'EAST MIDLANDS':'East Midlands', 'WEST MIDLANDS':'West Midlands','EAST':'East England', 'LONDON':'Greater London', 'SOUTH EAST':'South East', 'SOUTH WEST':'South West'}})

In [13]:
df_tax = df_tax.reset_index()

In [14]:
df_tax.drop(columns=['index', 'Geography1'], inplace=True)

In [15]:
df_tax


,Area Name,Band,Bungalow,Flat,Terraced,Semi-Detached,Detached
0,North East,All,"140,720","196,710","364,130","365,330","136,400"
1,North East,A,"58,850","162,100","276,110","158,280","1,570"
2,North East,B,"25,850","18,760","51,180","86,800","3,710"
3,North East,C,"27,970","9,730","21,490","90,210","28,220"
4,North East,D,"15,220","4,610","8,860","21,140","47,890"
...,...,...,...,...,...,...,...
76,South West,D,"97,030","27,110","50,010","87,910","127,200"
77,South West,E,"52,300","11,580","19,010","26,010","148,850"
78,South West,F,"17,260","4,220","6,910","9,760","92,050"
79,South West,G,"5,030","2,090","3,050","4,500","58,930"


In [16]:
def col_strip(x):
    return x.replace(',', '')
for column in ['Bungalow','Flat','Terraced','Semi-Detached','Detached']:
    df_tax[column] = df_tax[column].apply(col_strip)
df_tax = df_tax.astype({'Area Name':'str', 'Band':'str', 'Bungalow':'int32', 'Flat':'int32', 'Terraced':'int32', 'Semi-Detached':'int32', 'Detached':'int32'})

In [17]:
def percentage(column):
   if column[0] == 0:
      return 0
   return (column/column[0])*100

In [18]:
regions_df = []
types = []
types = ['Bungalow', 'Flat', 'Terraced', 'Semi-Detached', 'Detached']
for area in df_tax['Area Name'].unique():
    temp = (df_tax.loc[df_tax['Area Name']==area]).copy(deep=True).reset_index(drop=True)
    regions_df.append(temp)

In [19]:
def sum(a,b,c,d,e):
    return (a+b+c+d+e)/5

def sum2(a,b,c,d):
    return (a+b+c+d)/4

In [20]:
for i in range(len(regions_df)):
    for type in types:
        (regions_df[i])[type+'_%'] = percentage((regions_df[i])[type])
    regions_df[i]['All_Band_%'] = sum(regions_df[i]['Flat_%'], regions_df[i]['Terraced_%'], regions_df[i]['Bungalow_%'], regions_df[i]['Semi-Detached_%'], regions_df[i]['Detached_%'])
    regions_df[i]['All_Band_%_except_Flats'] = sum2(regions_df[i]['Terraced_%'], regions_df[i]['Bungalow_%'], regions_df[i]['Semi-Detached_%'], regions_df[i]['Detached_%'])

In [21]:
final = pd.DataFrame(columns=regions_df[0].columns)
for df in regions_df:
    final = pd.concat([final,df], ignore_index=True)
cols = list(final)
for i in range(2,len(cols)):
    final = final.astype({cols[i]:'float64'})

In [22]:
final.dtypes

Area Name                   object
Band                        object
Bungalow                   float64
Flat                       float64
Terraced                   float64
Semi-Detached              float64
Detached                   float64
Bungalow_%                 float64
Flat_%                     float64
Terraced_%                 float64
Semi-Detached_%            float64
Detached_%                 float64
All_Band_%                 float64
All_Band_%_except_Flats    float64
dtype: object

In [23]:
def sum(a,b,c,d,e):
    return (a+b+c+d+e)/5

def sum2(a,b,c,d):
    return (a+b+c+d)/4

In [24]:
final['All_Band_%'] = sum(final['Flat_%'], final['Terraced_%'], final['Bungalow_%'], final['Semi-Detached_%'], final['Detached_%'])
final['All_Band_%_except_Flats'] = sum2(final['Terraced_%'], final['Bungalow_%'], final['Semi-Detached_%'], final['Detached_%'])
final

,Area Name,Band,Bungalow,Flat,Terraced,Semi-Detached,Detached,Bungalow_%,Flat_%,Terraced_%,Semi-Detached_%,Detached_%,All_Band_%,All_Band_%_except_Flats
0,North East,All,140720.0,196710.0,364130.0,365330.0,136400.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,North East,A,58850.0,162100.0,276110.0,158280.0,1570.0,41.820637,82.405572,75.827314,43.325213,1.151026,48.905952,40.531048
2,North East,B,25850.0,18760.0,51180.0,86800.0,3710.0,18.369812,9.536882,14.055420,23.759341,2.719941,13.688279,14.726129
3,North East,C,27970.0,9730.0,21490.0,90210.0,28220.0,19.876350,4.946368,5.901738,24.692744,20.689150,15.221270,17.789995
4,North East,D,15220.0,4610.0,8860.0,21140.0,47890.0,10.815804,2.343551,2.433197,5.786549,35.109971,11.297815,13.536380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,South West,D,97030.0,27110.0,50010.0,87910.0,127200.0,29.714583,5.606568,8.030252,16.877856,26.443807,17.334613,20.266624
77,South West,E,52300.0,11580.0,19010.0,26010.0,148850.0,16.016415,2.394838,3.052491,4.993664,30.944659,11.480413,13.751807
78,South West,F,17260.0,4220.0,6910.0,9760.0,92050.0,5.285723,0.872730,1.109559,1.873824,19.136418,5.655651,6.851381
79,South West,G,5030.0,2090.0,3050.0,4500.0,58930.0,1.540393,0.432229,0.489747,0.863956,12.251050,3.115475,3.786287


In [25]:
final.to_csv('gb-tax-band-information.csv')